In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


In [21]:
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float


In [22]:
print(sentences)

['i like dog', 'i love coffee', 'i hate milk', 'you like cat', 'you love milk', 'you hate coffee']


In [23]:
print(" ".join(sentences))

i like dog i love coffee i hate milk you like cat you love milk you hate coffee


In [24]:
print(" ".join(sentences).split())

['i', 'like', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk', 'you', 'like', 'cat', 'you', 'love', 'milk', 'you', 'hate', 'coffee']


In [25]:
set(" ".join(sentences).split())

{'cat', 'coffee', 'dog', 'hate', 'i', 'like', 'love', 'milk', 'you'}

In [26]:
list(set(" ".join(sentences).split()))

['coffee', 'like', 'i', 'hate', 'milk', 'you', 'love', 'dog', 'cat']

In [27]:
word_list = list(set(" ".join(sentences).split()))
for i, w in enumerate(word_list):
    print(i)
    print(w)


0
coffee
1
like
2
i
3
hate
4
milk
5
you
6
love
7
dog
8
cat


In [28]:
"""
Word Processing
"""
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)



In [29]:

"""
TextRNN Parameter
"""
batch_size = len(sentences)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈

def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)



In [36]:

"""
TextLSTM
"""
class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.fc = nn.Linear(n_hidden, n_class)

    # self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    # self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    # self.Softmax = nn.Softmax(dim=1) # softmax 포함되어있다고 하지 않았나? CrossEntropyLoss에...

  def forward(self, hidden_and_cell, X):
    X = X.transpose(0, 1)
    # outputs, hidden = self.lstm(X, hidden_and_cell)
    # outputs = outputs[-1]  # 최종 예측 Hidden Layer
    # print(X.size())
    outputs, (h_n,c_n) = self.lstm(X, hidden_and_cell)
    print(outputs.size())
    print(outputs[-1])
    print(h_n[-1])
    # print(h_n.size())
    outputs = h_n[0,:,-1]  # 최종 예측 Hidden Layer
    # print(outputs.size())

    # model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    model = self.fc(outputs)  # 최종 예측 최종 출력 층
    
    return model
	


In [37]:

"""
Training
"""
model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model((hidden, cell), input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
cell = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model((hidden, cell), input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

torch.Size([2, 6, 5])
tensor([[ 0.1159,  0.0809,  0.1429, -0.1248,  0.1029],
        [-0.0422,  0.0822,  0.1058,  0.0103,  0.0065],
        [ 0.0354,  0.0551, -0.0029, -0.0357,  0.0555],
        [ 0.1614,  0.0713,  0.1267, -0.1456,  0.1389],
        [ 0.0401,  0.0729,  0.0845,  0.0020,  0.0436],
        [ 0.1048,  0.0454, -0.0096, -0.0501,  0.0903]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.1159,  0.0809,  0.1429, -0.1248,  0.1029],
        [-0.0422,  0.0822,  0.1058,  0.0103,  0.0065],
        [ 0.0354,  0.0551, -0.0029, -0.0357,  0.0555],
        [ 0.1614,  0.0713,  0.1267, -0.1456,  0.1389],
        [ 0.0401,  0.0729,  0.0845,  0.0020,  0.0436],
        [ 0.1048,  0.0454, -0.0096, -0.0501,  0.0903]],
       grad_fn=<SelectBackward0>)


/Users/goldenyoo/miniforge3/envs/mac_cpu/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x6 and 5x9)

In [ ]:
list(model.named_parameters())